In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torchvision import models
from torchvision.models.vgg import VGG
from sklearn.metrics import confusion_matrix
import pandas as pd
import scipy.misc
import random
import sys

if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

from matplotlib import pyplot as plt
import numpy as np
import time
import os

In [2]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [3]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [4]:
class UNet(nn.Module):
    def __init__(self, n_channels = 3, n_classes = 5, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [5]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels, 64, normalization=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1, bias=False)
        )

    def forward(self, img_A, img_B):
        # Concatenate image and condition image by channels to produce input
        img_input = torch.cat((img_A, img_B), 1)
        return self.model(img_input)

In [6]:
class UNetDown(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 4, 2, 1, bias=False)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_size))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class UNetUp(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [  nn.ConvTranspose2d(in_size, out_size, 4, 2, 1, bias=False),
                    nn.InstanceNorm2d(out_size),
                    nn.ReLU(inplace=True)]
        if dropout:
            layers.append(nn.Dropout(dropout))

        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)

        return x

class GeneratorUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(GeneratorUNet, self).__init__()

        self.down1 = UNetDown(in_channels, 64, normalize=False)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512, dropout=0.5)
        self.down5 = UNetDown(512, 512, dropout=0.5)
        self.down6 = UNetDown(512, 512, dropout=0.5)
        self.down7 = UNetDown(512, 512, dropout=0.5)
        self.down8 = UNetDown(512, 512, normalize=False, dropout=0.5)

        self.up1 = UNetUp(512, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5)
        self.up4 = UNetUp(1024, 512, dropout=0.5)
        self.up5 = UNetUp(1024, 256)
        self.up6 = UNetUp(512, 128)
        self.up7 = UNetUp(256, 64)


        self.final = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(128, out_channels, 4, padding=1),
            nn.Tanh()
        )


    def forward(self, x):
        # U-Net generator with skip connections from encoder to decoder
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)
        u1 = self.up1(d8, d7)
        u2 = self.up2(u1, d6)
        u3 = self.up3(u2, d5)
        u4 = self.up4(u3, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)

        return self.final(u7)

## Define training parameter

In [7]:
batch_size = 8
epochs     = 1000  #500
lr         = 0.01
momentum   = 0
w_decay    = 1e-8
step_size  = 45
gamma      = 0.8
n_class = 5
model_use = "pst_unet_gan"

## Define path, directory trainning environment

In [8]:
# get data
data_dir  = os.path.join("/media/arg_ws3/5E703E3A703E18EB/data/PST900_RGBT_Dataset/")
if not os.path.exists(data_dir):
    print("Data not found!")
# create dir for model
model_dir = os.path.join("/media/arg_ws3/5E703E3A703E18EB/research/subt_fcn/models", model_use)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
# create dir for score
score_dir = os.path.join("/media/arg_ws3/5E703E3A703E18EB/research/subt_fcn/scores", model_use)
if not os.path.exists(score_dir):
    os.makedirs(score_dir)

use_gpu = torch.cuda.is_available()
num_gpu = list(range(torch.cuda.device_count()))

#use_gpu = False
if use_gpu:
    ts = time.time()
    print("Finish cuda loading, time elapsed {}".format(time.time() - ts))

Finish cuda loading, time elapsed 7.152557373046875e-07


## GAN

In [9]:
fcn_temp = UNet(3,5)
discriminator = Discriminator(in_channels = n_class + 3)
discriminator_adv = Discriminator(in_channels = n_class*2)
fcn_model = UNet(3,5)
# Loss functions
criterion_GAN = torch.nn.MSELoss()
criterion_pixelwise = torch.nn.L1Loss()
criterion_adv = torch.nn.L1Loss()
patch = (1, 256//2**4, 256//2**4)
# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel = 100

if use_gpu:
    discriminator = discriminator.cuda()
    criterion_GAN.cuda()
    criterion_pixelwise.cuda()
    criterion_adv.cuda()
    fcn_model = fcn_model.cuda()
    fcn_temp = fcn_temp.cuda()
    fcn_temp = nn.DataParallel(fcn_temp, device_ids=num_gpu)
    discriminator_adv = discriminator_adv.cuda()
Tensor = torch.cuda.FloatTensor if use_gpu else torch.FloatTensor
discriminator.apply(weights_init_normal)
#fcn_model.apply(weights_init_normal)
discriminator_adv.apply(weights_init_normal)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.001, betas=(0.5, 0.999))
optimizer_adv_D = torch.optim.Adam(discriminator_adv.parameters(), lr=0.001, betas=(0.5, 0.999))

scheduler_d = lr_scheduler.StepLR(optimizer_D, step_size = step_size+10, gamma = gamma)
scheduler_adv = lr_scheduler.StepLR(optimizer_adv_D, step_size = step_size+10, gamma = gamma)  

In [10]:
means     = np.array([103.939, 116.779, 123.68]) / 255. # mean of three channels in the order of BGR
h, w      = 256, 256
val_h     = h
val_w     = w

class product_dataset(Dataset):

    def __init__(self, root, phase, n_class=n_class, flip_rate=0.):
        data_dir = os.path.join(root, phase)
        self.img_list = os.listdir(os.path.join(data_dir, 'rgb'))
        self.rgb_dir = os.path.join(data_dir, 'rgb')
        self.label_dir = os.path.join(data_dir, 'labels')
        self.means     = means
        self.n_class   = n_class
        self.flip_rate = flip_rate
        if phase == 'train':
            self.flip_rate = 0.5

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        idx = idx % len(self.img_list)
        img = cv2.imread(os.path.join(self.rgb_dir, self.img_list[idx]),cv2.IMREAD_UNCHANGED)
        label = cv2.imread(os.path.join(self.label_dir, self.img_list[idx]), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_CUBIC)
        label = cv2.resize(label, (256, 256), interpolation=cv2.INTER_CUBIC)
        origin_img = img
        if random.random() < self.flip_rate:
            img   = np.fliplr(img)
            label = np.fliplr(label)

        # reduce mean
        img = img[:, :, ::-1]  # switch to BGR
        
        img = np.transpose(img, (2, 0, 1)) / 255.
        #img = np.transpose(img, (2, 0, 1))
        #img[0] -= self.means[0]
        #img[1] -= self.means[1]
        #img[2] -= self.means[2]
        label[label > 4] = 0
        
        #print(label.min(), "++++",  label.max())
        #if label.max() == 5:
        #    print(self.img_list[idx])
        
        # convert to tensor
        img = torch.from_numpy(img.copy()).float()
        label = torch.from_numpy(label.copy()).float()
        #tf = torchvision.transforms.Compose([torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ])
        
        #img = tf(img)
        
        
        # create one-hot encoding
        h, w = label.size()
        target = torch.zeros(self.n_class, h, w)
        
        for i in range(n_class):
            target[i][label == i] = 1
        
        #target[0][label == 0] = 1
        #print(np.unique(label))
        
 
        sample = {'X': img, 'Y': target, 'l': label, 'origin': origin_img}

        return sample

## Define dataloader and optimizer

In [11]:
# initial dataloader for trainning and validation
train_data = product_dataset(data_dir, phase = 'train')
val_data   = product_dataset(data_dir, phase = 'test', flip_rate = 0)
dataloader = DataLoader(train_data, batch_size = batch_size, shuffle=True, num_workers = 0)
val_loader = DataLoader(val_data, batch_size = 1, num_workers = 0)

dataiter = iter(val_loader)

# define loss function
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()
#optimizer = optim.RMSprop(fcn_model.parameters(), lr = lr, momentum = momentum, weight_decay = w_decay)
optimizer = optim.RMSprop(fcn_model.parameters(), lr=lr, weight_decay=1e-8)
# decay LR by a factor of 0.5 every step_size = 50 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size = step_size, gamma = gamma)  

In [12]:
len(dataiter)

288

## Train

In [13]:
def train():
    for epoch in range(epochs):
        fcn_model.train()
        scheduler.step()
        scheduler_d.step()
        scheduler_adv.step()
        configs    = "FCNs_batch{}_epoch{}_RMSprop_lr{}"\
            .format(batch_size, epoch, lr)
        model_path = os.path.join(model_dir, configs)
        
        ts = time.time()
        for iter, batch in enumerate(dataloader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = Variable(batch['X'].cuda())
                labels = Variable(batch['Y'].cuda())
                targets = Variable(batch['l'].cuda())
            else:
                inputs, labels = Variable(batch['X']), Variable(batch['Y'])
                targets = Variable(batch['l'])

            valid = Variable(Tensor(np.ones((labels.size(0), *patch))), requires_grad=False)
            fake = Variable(Tensor(np.zeros((labels.size(0), *patch))), requires_grad=False)
                
            outputs = fcn_model(inputs)
            temp = fcn_temp(inputs).detach()
            # GAN loss
            pred_fake = discriminator(outputs, inputs)
            loss_GAN = criterion_GAN(pred_fake, valid)
            # ADV loss
            #pred_adv = discriminator_adv(outputs, temp)
            #loss_adv = criterion_adv(pred_adv, valid)
            # Pixel-wise loss
            
            #outputs = outputs.long()
            targets = targets.long()
            
            loss_pixel = criterion(outputs, targets)
            #loss_G = 0.005 * loss_GAN + loss_pixel + 0.005 * loss_adv
            #loss_G = loss_pixel + 0.01 * loss_adv
            loss_G = 0.001 * loss_GAN + loss_pixel
            loss_G.backward()
            optimizer.step()

            # ---------------------
            #  Train Discriminator
            # ---------------------

            optimizer_adv_D.zero_grad()
            optimizer_D.zero_grad()

            # Real loss
            pred_real = discriminator(labels, inputs)
            loss_real = criterion_GAN(pred_real, valid)
            #pred_adv_real = discriminator_adv(outputs.detach(),labels)
            #loss_adv_real = criterion_adv(pred_adv_real, valid)

            # Fake loss
            pred_fake = discriminator(outputs.detach(), inputs)
            loss_fake = criterion_GAN(pred_fake, fake)
            #pred_adv_fake = discriminator_adv(outputs.detach(), temp)
            #loss_adv_fake = criterion_adv(pred_adv_fake, fake)

            # Total loss
            #loss_D = 0.25*0.5 * (loss_real + loss_fake) + 0.25*0.5 * (loss_adv_real + loss_adv_fake)
            #loss_D = 0.5 * (loss_adv_real + loss_adv_fake)
            loss_D = 0.5 * (loss_real + loss_fake)

            loss_D.backward()
            optimizer_D.step()
            optimizer_adv_D.step()
            
            #if iter % 80 == 0:
            #    fcn_temp.load_state_dict(fcn_model.state_dict())
            #    print("Load temp")
            
            if iter % 10 == 0:
                #print("epoch{}, iter{}, loss_G: {}, loss_D: {}".format(epoch+1, iter, loss_G.item(), loss_D.item()))
                print("epoch{}, iter{}, loss_G: {}".format(epoch+1, iter, loss_G.item()))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        if epoch % 20 == 0:
            torch.save(fcn_model.state_dict(),model_path + '.pkl')

        val(epoch)

## Evaluation

In [14]:
def val(epoch):
    fcn_model.eval()
    TP = np.zeros(n_class-1, dtype = np.float128)
    FN = np.zeros(n_class-1, dtype = np.float128)
    FP = np.zeros(n_class-1, dtype = np.float128)
    total_ious = []
    pixel_accs = []
    for iter, batch in enumerate(val_loader):
        if use_gpu:
            inputs = Variable(batch['X'].cuda())
        else:
            inputs = Variable(batch['X'])

        output = fcn_model(inputs)
        output = output.data.cpu().numpy()

        N, _, h, w = output.shape
        pred = output.transpose(0, 2, 3, 1).reshape(-1, n_class).argmax(axis=1).reshape(N, h, w)

        target = batch['l'].cpu().numpy().reshape(N, h, w)
        for p, t in zip(pred, target):
            pixel_accs.append(pixel_acc(p, t))
            _TP, _FN, _FP =  analysis(p, t, h, w)
            TP += _TP[1:n_class]
            FN += _FN[1:n_class]
            FP += _FP[1:n_class]
            
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    ious = TP / (TP + FN + FP)
    fscore = 2*TP / (2*TP + FN + FP)
    total_ious = np.array(total_ious).T  # n_class * val_len
    pixel_accs = np.array(pixel_accs).mean()
    
    #print("epoch{}, pix_acc: {}, meanIoU: {}, IoUs: {}, recall: {}, precision: {}, fscore: {}"\
    #      .format(epoch, pixel_accs, np.nanmean(ious), ious, recall, precision, fscore))
    print("epoch{}, meanIoU: {}".format(epoch, np.nanmean(ious)))
    
    f1 = open(score_dir + "/cls_acc_log.txt","a+")
    f1.write('epoch:'+ str(epoch) + ', pix_acc: ' + str(pixel_accs) + '\n' )
    f2 = open(score_dir + "/cls_iou_log.txt","a+")
    f2.write('epoch:'+ str(epoch) + ', class ious: ' + str(ious) + '\n' )
    f3 = open(score_dir + "/mean_iou_log.txt","a+")
    f3.write('epoch:'+ str(epoch) + ', mean IoU: ' + str(np.nanmean(ious)) + '\n' ) 
    f4 = open(score_dir + "/recall_log.txt","a+")
    f4.write('epoch:'+ str(epoch) + ', class recall: ' + str(recall) + '\n' )
    f5 = open(score_dir + "/precision_log.txt","a+")
    f5.write('epoch:'+ str(epoch) + ', class precision: ' + str(precision) + '\n' )    
    f6 = open(score_dir + "/fscore_log.txt","a+")
    f6.write('epoch:'+ str(epoch) + ', class fscore: ' + str(fscore) + '\n' )  
    f7 = open(score_dir + "/mean_fscore_log.txt","a+")
    f7.write('epoch:'+ str(epoch) + ', mean fscore: ' + str(np.nanmean(fscore)) + '\n' )
    f8 = open(score_dir + "/mean_precision_log.txt","a+")
    f8.write('epoch:'+ str(epoch) + ', mean precision: ' + str(np.nanmean(precision)) + '\n' ) 
    f9 = open(score_dir + "/mean_recall_log.txt","a+")
    f9.write('epoch:'+ str(epoch) + ', mean recall: ' + str(np.nanmean(recall)) + '\n' ) 
    

def analysis(pred, target, h, w):
    # TP, FN, FP, TN
    TP = np.zeros(n_class, dtype = np.float128)
    FN = np.zeros(n_class, dtype = np.float128)
    FP = np.zeros(n_class, dtype = np.float128)

    target = target.reshape(h * w)
    pred = pred.reshape(h * w)

    con_matrix = confusion_matrix(target, pred,labels = np.arange(0,n_class,1))
    con_matrix[0][0] = 0
    for i in range(0, n_class):
        for j in range(0, n_class):
            if i == j:
                TP[i] += con_matrix[i][j]
            if i != j:
                FP[j] += con_matrix[i][j]
                FN[i] += con_matrix[i][j]
    return TP, FN, FP
                
def pixel_acc(pred, target):
    correct = (pred == target).sum()
    total   = (target == target).sum()
    return correct / total

In [15]:
train()

epoch1, iter0, loss_G: 1.5832617282867432
epoch1, iter10, loss_G: 0.13086655735969543
epoch1, iter20, loss_G: 0.375337690114975
epoch1, iter30, loss_G: 0.05953691527247429
epoch1, iter40, loss_G: 0.12719787657260895
epoch1, iter50, loss_G: 0.1139308512210846
epoch1, iter60, loss_G: 0.12109367549419403
epoch1, iter70, loss_G: 0.17914709448814392
Finish epoch 0, time elapsed 50.77822303771973


/home/arg_ws3/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide


epoch0, meanIoU: 0.08037713363988733
epoch2, iter0, loss_G: 0.09370297193527222
epoch2, iter10, loss_G: 0.08759605139493942
epoch2, iter20, loss_G: 0.11387620866298676
epoch2, iter30, loss_G: 0.12081034481525421
epoch2, iter40, loss_G: 0.11949840188026428
epoch2, iter50, loss_G: 0.06092185899615288
epoch2, iter60, loss_G: 0.13812865316867828
epoch2, iter70, loss_G: 0.056781869381666183
Finish epoch 1, time elapsed 51.35724639892578
epoch1, meanIoU: 0.09952653658794719
epoch3, iter0, loss_G: 0.14531804621219635
epoch3, iter10, loss_G: 0.03869002312421799
epoch3, iter20, loss_G: 0.09471235424280167
epoch3, iter30, loss_G: 0.12114497274160385
epoch3, iter40, loss_G: 0.054278235882520676
epoch3, iter50, loss_G: 0.06877270340919495
epoch3, iter60, loss_G: 0.07918036729097366
epoch3, iter70, loss_G: 0.08219030499458313
Finish epoch 2, time elapsed 49.83029866218567
epoch2, meanIoU: 0.12612418614689824
epoch4, iter0, loss_G: 0.17041011154651642
epoch4, iter10, loss_G: 0.13280948996543884
epoc

epoch20, iter50, loss_G: 0.020815428346395493
epoch20, iter60, loss_G: 0.04940075799822807
epoch20, iter70, loss_G: 0.058284953236579895
Finish epoch 19, time elapsed 49.89114737510681
epoch19, meanIoU: 0.350105185189853
epoch21, iter0, loss_G: 0.035452112555503845
epoch21, iter10, loss_G: 0.05800146982073784
epoch21, iter20, loss_G: 0.07434090971946716
epoch21, iter30, loss_G: 0.02204212173819542
epoch21, iter40, loss_G: 0.07324602454900742
epoch21, iter50, loss_G: 0.06174388900399208
epoch21, iter60, loss_G: 0.03007495030760765
epoch21, iter70, loss_G: 0.047844480723142624
Finish epoch 20, time elapsed 50.063217639923096
epoch20, meanIoU: 0.31011532082247184
epoch22, iter0, loss_G: 0.054323215037584305
epoch22, iter10, loss_G: 0.05697951093316078
epoch22, iter20, loss_G: 0.07268402725458145
epoch22, iter30, loss_G: 0.03060387261211872
epoch22, iter40, loss_G: 0.13169439136981964
epoch22, iter50, loss_G: 0.023367492482066154
epoch22, iter60, loss_G: 0.03853511065244675
epoch22, iter70

Finish epoch 37, time elapsed 49.6611545085907
epoch37, meanIoU: 0.22997680503218426
epoch39, iter0, loss_G: 0.03223536163568497
epoch39, iter10, loss_G: 0.021879373118281364
epoch39, iter20, loss_G: 0.04760091006755829
epoch39, iter30, loss_G: 0.02005372941493988
epoch39, iter40, loss_G: 0.05794373154640198
epoch39, iter50, loss_G: 0.03972313180565834
epoch39, iter60, loss_G: 0.014633162878453732
epoch39, iter70, loss_G: 0.14167700707912445
Finish epoch 38, time elapsed 49.659963846206665
epoch38, meanIoU: 0.35944059650275983
epoch40, iter0, loss_G: 0.02665187418460846
epoch40, iter10, loss_G: 0.04723589867353439
epoch40, iter20, loss_G: 0.09774433821439743
epoch40, iter30, loss_G: 0.06367146968841553
epoch40, iter40, loss_G: 0.015369586646556854
epoch40, iter50, loss_G: 0.03255986049771309
epoch40, iter60, loss_G: 0.04220423847436905
epoch40, iter70, loss_G: 0.016630521044135094
Finish epoch 39, time elapsed 49.740403175354004
epoch39, meanIoU: 0.37418820445459916
epoch41, iter0, los

...

epoch342, iter50, loss_G: 0.0051916190423071384
epoch342, iter60, loss_G: 0.0035877316258847713
epoch342, iter70, loss_G: 0.004499139729887247
Finish epoch 341, time elapsed 50.95083928108215
epoch341, meanIoU: 0.48393247749588786
epoch343, iter0, loss_G: 0.0032446524128317833
epoch343, iter10, loss_G: 0.004186130594462156
epoch343, iter20, loss_G: 0.0028126020915806293
epoch343, iter30, loss_G: 0.004880272317677736
epoch343, iter40, loss_G: 0.003612726926803589
epoch343, iter50, loss_G: 0.004478448070585728
epoch343, iter60, loss_G: 0.005212657153606415
epoch343, iter70, loss_G: 0.002816030289977789
Finish epoch 342, time elapsed 50.895058393478394
epoch342, meanIoU: 0.5045610904748385
epoch344, iter0, loss_G: 0.002415054477751255
epoch344, iter10, loss_G: 0.006689831614494324
epoch344, iter20, loss_G: 0.005570290610194206
epoch344, iter30, loss_G: 0.0038384562358260155
epoch344, iter40, loss_G: 0.005790464114397764
epoch344, iter50, loss_G: 0.003339609131217003
epoch344, iter60, loss

epoch360, iter20, loss_G: 0.005495550110936165
epoch360, iter30, loss_G: 0.00282463850453496
epoch360, iter40, loss_G: 0.005308697000145912
epoch360, iter50, loss_G: 0.003786996006965637
epoch360, iter60, loss_G: 0.0025336905382573605
epoch360, iter70, loss_G: 0.0031056972220540047
Finish epoch 359, time elapsed 50.476452350616455
epoch359, meanIoU: 0.49830399685597004
epoch361, iter0, loss_G: 0.004027480259537697
epoch361, iter10, loss_G: 0.003878762014210224
epoch361, iter20, loss_G: 0.004921766929328442
epoch361, iter30, loss_G: 0.002386088017374277
epoch361, iter40, loss_G: 0.0052261375822126865
epoch361, iter50, loss_G: 0.004000352695584297
epoch361, iter60, loss_G: 0.004062599036842585
epoch361, iter70, loss_G: 0.00522702606394887
Finish epoch 360, time elapsed 49.90060997009277
epoch360, meanIoU: 0.48918910982468033
epoch362, iter0, loss_G: 0.0050251418724656105
epoch362, iter10, loss_G: 0.003608567174524069
epoch362, iter20, loss_G: 0.004357068333774805
epoch362, iter30, loss_G

epoch376, meanIoU: 0.49524424985931836
epoch378, iter0, loss_G: 0.005269290879368782
epoch378, iter10, loss_G: 0.0031966776587069035
epoch378, iter20, loss_G: 0.0025036893784999847
epoch378, iter30, loss_G: 0.003544462379068136
epoch378, iter40, loss_G: 0.0031422050669789314
epoch378, iter50, loss_G: 0.004569888114929199
epoch378, iter60, loss_G: 0.003887442173436284
epoch378, iter70, loss_G: 0.003204832784831524
Finish epoch 377, time elapsed 51.13860821723938
epoch377, meanIoU: 0.49099648558561937
epoch379, iter0, loss_G: 0.006921946536749601
epoch379, iter10, loss_G: 0.0057769655250012875
epoch379, iter20, loss_G: 0.003662331495434046
epoch379, iter30, loss_G: 0.0028827611822634935
epoch379, iter40, loss_G: 0.0036351634189486504
epoch379, iter50, loss_G: 0.0041370815597474575
epoch379, iter60, loss_G: 0.0035904645919799805
epoch379, iter70, loss_G: 0.0025294667575508356
Finish epoch 378, time elapsed 50.90567088127136
epoch378, meanIoU: 0.5108631735220839
epoch380, iter0, loss_G: 0.

epoch395, iter60, loss_G: 0.003536699805408716
epoch395, iter70, loss_G: 0.00360447377897799
Finish epoch 394, time elapsed 50.66074514389038
epoch394, meanIoU: 0.5059595566535559
epoch396, iter0, loss_G: 0.00468162726610899
epoch396, iter10, loss_G: 0.0033581051975488663
epoch396, iter20, loss_G: 0.005380350165069103
epoch396, iter30, loss_G: 0.0030922074802219868
epoch396, iter40, loss_G: 0.0036865752190351486
epoch396, iter50, loss_G: 0.0025511817075312138
epoch396, iter60, loss_G: 0.003800375387072563
epoch396, iter70, loss_G: 0.00564462598413229
Finish epoch 395, time elapsed 50.935920000076294
epoch395, meanIoU: 0.5064992880515095
epoch397, iter0, loss_G: 0.0029663590248674154
epoch397, iter10, loss_G: 0.004136277362704277
epoch397, iter20, loss_G: 0.0061484044417738914
epoch397, iter30, loss_G: 0.0030138425063341856
epoch397, iter40, loss_G: 0.0029599692206829786
epoch397, iter50, loss_G: 0.004206025507301092
epoch397, iter60, loss_G: 0.003007963765412569
epoch397, iter70, loss_

KeyboardInterrupt: 

## Prediction Result

In [ ]:
val_data   = product_dataset(data_dir, phase = 'test', flip_rate = 0)
val_loader = DataLoader(val_data, batch_size = 1, num_workers = 0)
dataiter = iter(val_loader)

In [ ]:
def prediction(model_name):
    
    # load pretrain models
    state_dict = torch.load(os.path.join(model_name))
    fcn_model.load_state_dict(state_dict)
    
    batch = dataiter.next()
    if use_gpu:
        inputs = Variable(batch['X'].cuda())
    else:
        inputs = Variable(batch['X'])
    img    = batch['origin'] 
    label  = batch['l']
    
    #print(img.shape, inputs.shape)
    #print(inputs[0])
    
    output = fcn_model(inputs)
    output = output.data.cpu().numpy()

    N, _, h, w = output.shape
    pred = output.transpose(0, 2, 3, 1).reshape(-1, n_class).argmax(axis = 1).reshape(N, h, w)

    # show images
    plt.figure(figsize = (10, 12))
    img = img.numpy()
    for i in range(N):
        img[i] = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
        plt.subplot(N, 3, i*3 + 1)
        plt.title("origin_img")
        plt.imshow(img[i])
        #print(np.unique(_img[i]))

        plt.subplot(N, 3, i*3 + 2)
        plt.title("label_img")
        plt.imshow(label[i],cmap = "nipy_spectral",vmin = 0, vmax = n_class - 1)

        plt.subplot(N, 3, i*3 + 3)
        plt.title("prediction")
        plt.imshow(pred[i],cmap = "nipy_spectral",vmin = 0, vmax = n_class - 1)
    y = pred[0].astype(np.uint8)
    #y = cv2.cvtColor(y, cv2.COLOR_GRAY2RGB)
    plt.imsave("/media/arg_ws3/5E703E3A703E18EB/research/subt_fcn/models/pst/b_6.jpg", y, cmap='nipy_spectral',vmin = 0, vmax = n_class - 1)
    #cv2.imwrite("/media/arg_ws3/5E703E3A703E18EB/research/subt_fcn/models/pst/a.jpg", y)
    plt.show()

In [ ]:
prediction("/media/arg_ws3/5E703E3A703E18EB/research/subt_fcn/models/pst_unet_gan/FCNs_batch8_epoch80_RMSprop_lr0.00015.pkl")